In [4]:
import pandas as pd
from config import (
    date_part,
    dbusername,
    dbpassword,
    dbhost,
    dbport,
    dbname,
)


In [ ]:
#Load up the original parquet data source
df = pd.read_parquet (f'data/cleaned/cleaned_trips_{date_part}.parquet')
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,cbd_congestion_fee
0,1,2025-07-01 00:29:37,2025-07-01 00:45:30,1,7.30,1,N,138,74,1,29.60,7.75,0.5,9.00,6.94,1.0,54.79,0.0,1.75,0.00
1,1,2025-07-01 00:23:28,2025-07-01 01:07:44,1,17.70,2,N,132,142,1,70.00,4.25,0.5,5.00,0.00,1.0,80.75,2.5,1.75,0.00
2,2,2025-07-01 00:53:50,2025-07-01 01:27:12,1,9.98,1,N,138,48,1,43.60,6.00,0.5,10.87,0.00,1.0,66.97,2.5,1.75,0.75
3,2,2025-07-01 00:58:49,2025-07-01 01:15:55,1,10.27,1,N,138,229,1,38.70,6.00,0.5,14.10,6.94,1.0,72.24,2.5,1.75,0.75
4,2,2025-07-01 00:09:22,2025-07-01 00:23:54,1,2.94,1,N,211,97,1,17.00,1.00,0.5,3.00,0.00,1.0,25.75,2.5,0.00,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3898956,1,2025-07-31 23:23:43,2025-07-31 23:39:07,<NA>,3.70,<NA>,None,92,83,0,17.52,0.00,0.5,0.00,0.00,1.0,19.02,NaN,NaN,0.00
3898958,2,2025-07-31 23:43:43,2025-07-31 23:47:49,<NA>,0.64,<NA>,None,100,170,0,6.39,0.00,0.5,0.00,0.00,1.0,11.14,NaN,NaN,0.75
3898960,2,2025-07-31 23:08:32,2025-07-31 23:34:06,<NA>,7.80,<NA>,None,151,79,0,32.53,0.00,0.5,0.00,0.00,1.0,37.28,NaN,NaN,0.75
3898961,2,2025-07-31 23:13:56,2025-07-31 23:25:09,<NA>,2.09,<NA>,None,148,170,0,12.17,0.00,0.5,0.00,0.00,1.0,16.92,NaN,NaN,0.75


In [ ]:
#Renaming some of the columns for better consistency 
df.rename(columns={'tpep_pickup_datetime': 'pickup_datetime'}, inplace=True)
df.rename(columns={'tpep_dropoff_datetime': 'dropoff_datetime'}, inplace=True)
df.rename(columns={'trip_distance': 'trip_distance_miles'}, inplace=True)

In [ ]:
#Putting the column names in lowercase for compatability with PostGre
df.rename(columns={'VendorID': 'vendorid'}, inplace=True)
df.rename(columns={'RatecodeID': 'ratecodeid'}, inplace=True)
df.rename(columns={'PULocationID': 'pickup_locid'}, inplace=True)
df.rename(columns={'DOLocationID': 'dropoff_locid'}, inplace=True)
df.rename(columns={'Airport_fee': 'airport_fee'}, inplace=True)

In [ ]:
#Removing the number prefex for payment type and replacing them with string value according to yellow trips data dictionary
df['payment_type'] = df['payment_type'].astype('float64')
df.replace({'payment_type': {0: 'FlexFare trip'}}, inplace=True)
df.replace({'payment_type': {1: 'Card'}}, inplace=True)
df.replace({'payment_type': {2: 'Cash'}}, inplace=True)
df.replace({'payment_type': {3: 'No Charge'}}, inplace=True)
df.replace({'payment_type': {4: 'Dispute'}}, inplace=True)
df.replace({'payment_type': {5: 'Unknown'}}, inplace=True)
df.replace({'payment_type': {6: 'Void'}}, inplace=True)

In [ ]:
#Removing dates outside of the month timeframe
dates_to_drop = ['2009-01-01', '2025-06-30']
dates_to_drop = [pd.to_datetime(date).date() for date in dates_to_drop]

df = df[~df['pickup_datetime'].dt.date.isin(dates_to_drop)]

In [ ]:
#Reloading new dataframe for final checks
df

,vendorID,pickup_datetime,dropoff_datetime,passenger_count,trip_distance_miles,ratecodeID,store_and_fwd_flag,pickup_locID,dropoff_locID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,cbd_congestion_fee
0,1,2025-07-01 00:29:37,2025-07-01 00:45:30,1,7.30,1,N,138,74,Card,29.60,7.75,0.5,9.00,6.94,1.0,54.79,0.0,1.75,0.00
1,1,2025-07-01 00:23:28,2025-07-01 01:07:44,1,17.70,2,N,132,142,Card,70.00,4.25,0.5,5.00,0.00,1.0,80.75,2.5,1.75,0.00
2,2,2025-07-01 00:53:50,2025-07-01 01:27:12,1,9.98,1,N,138,48,Card,43.60,6.00,0.5,10.87,0.00,1.0,66.97,2.5,1.75,0.75
3,2,2025-07-01 00:58:49,2025-07-01 01:15:55,1,10.27,1,N,138,229,Card,38.70,6.00,0.5,14.10,6.94,1.0,72.24,2.5,1.75,0.75
4,2,2025-07-01 00:09:22,2025-07-01 00:23:54,1,2.94,1,N,211,97,Card,17.00,1.00,0.5,3.00,0.00,1.0,25.75,2.5,0.00,0.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3898956,1,2025-07-31 23:23:43,2025-07-31 23:39:07,<NA>,3.70,<NA>,None,92,83,FlexFare trip,17.52,0.00,0.5,0.00,0.00,1.0,19.02,NaN,NaN,0.00
3898958,2,2025-07-31 23:43:43,2025-07-31 23:47:49,<NA>,0.64,<NA>,None,100,170,FlexFare trip,6.39,0.00,0.5,0.00,0.00,1.0,11.14,NaN,NaN,0.75
3898960,2,2025-07-31 23:08:32,2025-07-31 23:34:06,<NA>,7.80,<NA>,None,151,79,FlexFare trip,32.53,0.00,0.5,0.00,0.00,1.0,37.28,NaN,NaN,0.75
3898961,2,2025-07-31 23:13:56,2025-07-31 23:25:09,<NA>,2.09,<NA>,None,148,170,FlexFare trip,12.17,0.00,0.5,0.00,0.00,1.0,16.92,NaN,NaN,0.75


In [ ]:
#Gathering column names and data types
df.info(all)

<class 'pandas.core.frame.DataFrame'>
Index: 3651868 entries, 0 to 3898962
Data columns (total 20 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   vendorID               int32         
 1   pickup_datetime        datetime64[us]
 2   dropoff_datetime       datetime64[us]
 3   passenger_count        Int64         
 4   trip_distance_miles    float64       
 5   ratecodeID             Int64         
 6   store_and_fwd_flag     object        
 7   pickup_locID           int32         
 8   dropoff_locID          int32         
 9   payment_type           object        
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee            floa

In [ ]:
#Authenticating connection to PostGre and importing data into master_table
from sqlalchemy import create_engine
import psycopg2

engine = create_engine(f'postgresql://{dbusername}:{dbpassword}@{dbhost}:{dbport}/{dbname}')

df.to_sql(f'master_table_{date_part}', engine)

868